In [ ]:
# # !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

In [ ]:
# !kaggle datasets download -d salader/dogs-vs-cats

 99% 1.06G/1.06G [00:07<00:00, 141MB/s]
100% 1.06G/1.06G [00:07<00:00, 162MB/s]


In [1]:
# import zipfile
# zip_ref = zipfile.ZipFile('/input/content/monkeypox.zip', 'r')
# zip_ref.extractall('/kaggle/working/monkeypox')
# zip_ref.close()

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import keras_tuner as kt
from sklearn.metrics import roc_auc_score, confusion_matrix
import seaborn as sns

2025-07-05 11:25:58.096630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751714758.278903      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751714758.328103      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
conv_base = InceptionV3(
    weights='imagenet',
    include_top = False,
    input_shape=(224,224,3)
)

I0000 00:00:1751714772.550278      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

In [4]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,  # Reduce learning rate by a factor of 0.2
    patience=3,  # Wait 3 epochs before reducing
    min_lr=1e-6  # Minimum learning rate
)

In [5]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define data augmentation as a keras Sequential model
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),  # Increase rotation
    layers.RandomZoom(0.3),
    layers.RandomTranslation(0.2, 0.2),
    layers.RandomContrast(0.2),  # Add contrast adjustment
    layers.RandomBrightness(0.2)  # Add brightness adjustment
])
# Load datasets from your directories (with your exact paths)
train_ds = keras.utils.image_dataset_from_directory(
    directory='/kaggle/input/content/Fold1/Fold1/Fold1/Train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/kaggle/input/content/Fold1/Fold1/Fold1/Val',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224)
)

# Function to apply augmentation and normalization to training data
def process_train(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    image = data_augmentation(image)            # Apply augmentation
    return image, label

# Function to normalize validation data only
def process_val(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image, label

# Apply the processing functions
train_ds = train_ds.map(process_train, num_parallel_calls=tf.data.AUTOTUNE)
validation_ds = validation_ds.map(process_val, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetch for performance
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
validation_ds = validation_ds.prefetch(tf.data.AUTOTUNE)

Found 2142 files belonging to 2 classes.
Found 420 files belonging to 2 classes.


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
import keras_tuner as kt
import json
import pickle
import shutil

# Create output directory for autosaving
output_dir = '/kaggle/working/inceptionv3_models/'
os.makedirs(output_dir, exist_ok=True)

# Assuming conv_base, train_ds, validation_ds are defined
# Unfreeze top 30 layers of InceptionV3
conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in conv_base.layers[-30:]:
    layer.trainable = True

# Custom callback to save model weights and trial data after each trial
class AutosaveCallback(tf.keras.callbacks.Callback):
    def __init__(self, output_dir, tuner):
        super().__init__()
        self.output_dir = output_dir
        self.tuner = tuner
    
    def on_epoch_end(self, epoch, logs=None):
        # Save trial model weights and data at the end of each epoch
        trial_id = self.tuner.oracle.ongoing_trial.trial_id
        trial_model = self.model
        trial_model.save_weights(os.path.join(self.output_dir, f'trial_{trial_id}_epoch_{epoch+1}_weights.weights.h5'))
        
        # Save trial hyperparameters
        trial_hps = self.tuner.oracle.get_trial(trial_id).hyperparameters.values
        with open(os.path.join(self.output_dir, f'trial_{trial_id}_hyperparameters.json'), 'w') as f:
            json.dump(trial_hps, f, indent=4)
        
        # Save trial metrics
        trial_metrics = self.tuner.oracle.get_trial(trial_id).metrics.get_history()
        with open(os.path.join(self.output_dir, f'trial_{trial_id}_metrics.pkl'), 'wb') as f:
            pickle.dump(trial_metrics, f)

# Define model-building function
def build_model(hp):
    model = Sequential()
    model.add(conv_base)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(
        units=hp.Int('units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Initialize Keras Tuner with checkpointing
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    directory=output_dir + 'tuner_dir',
    project_name='inceptionv3_tuning',
    overwrite=False  # Resume from checkpoint if exists
)

# Define callbacks for autosaving
autosave_callback = AutosaveCallback(output_dir, tuner)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(output_dir, 'checkpoint_best_model.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

# Perform hyperparameter search with autosaving
tuner.search(
    train_ds,
    validation_data=validation_ds,
    epochs=20,
    callbacks=[autosave_callback, checkpoint_callback]
)

# Save best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save(os.path.join(output_dir, 'best_model.keras'))
best_model.save_weights(os.path.join(output_dir, 'best_model_weights.weights.h5'))

# Save best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
with open(os.path.join(output_dir, 'best_hyperparameters.json'), 'w') as f:
    json.dump(best_hps.values, f, indent=4)

# Save trials summary
trials_summary = [
    {
        'trial_id': trial.trial_id,
        'hyperparameters': trial.hyperparameters.values,
        'score': trial.score,
        'status': trial.status
    } for trial in tuner.oracle.trials.values()
]
with open(os.path.join(output_dir, 'trials_summary.json'), 'w') as f:
    json.dump(trials_summary, f, indent=4)

# Save trials history for top 5 trials
best_trials = tuner.oracle.get_best_trials(num_trials=5)
trials_history = {trial.trial_id: trial.metrics.get_history() for trial in best_trials}
with open(os.path.join(output_dir, 'trials_history.pkl'), 'wb') as f:
    pickle.dump(trials_history, f)

# Create ZIP archive for backup
shutil.make_archive('/kaggle/working/inceptionv3_models', 'zip', output_dir)
print("All files saved in autosave mode in", output_dir)
print("ZIP archive created at /kaggle/working/inceptionv3_models.zip")

Trial 2 Complete [00h 02m 14s]

Best val_accuracy So Far: None
Total elapsed time: 00h 04m 45s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
384               |256               |units
0.4               |0.2               |dropout
0.00035972        |5.4318e-05        |learning_rate
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


In [7]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3



# Unfreeze top 30 layers of InceptionV3
for layer in conv_base.layers[-30:]:
    layer.trainable = True

# Freeze all layers initially
# conv_base.trainable = False

def build_model(hp):
    model = Sequential()
    model.add(conv_base)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(
        units=hp.Int('units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    directory='tuner_dir',
    project_name='inceptionv3_tuning'
)
tuner.search(train_ds, validation_data=validation_ds, epochs=20)
best_model = tuner.get_best_models(num_models=1)[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |units
0.2               |0.2               |dropout
0.00023177        |0.00023177        |learning_rate
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


KeyboardInterrupt: 

In [ ]:
# Optional: Get the best hyperparameters for reference
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
# Create output directory
output_dir = '/kaggle/working/inceptionv3_models/'
os.makedirs(output_dir, exist_ok=True)
best_model.save(os.path.join(output_dir, 'best_model.keras'))
best_model.save_weights(os.path.join(output_dir, 'best_model_weights.weights.h5'))
print(f"Best units: {best_hps.get('units')}")
print(f"Best dropout: {best_hps.get('dropout')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

# Define callbacks for further training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_finetuned_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

# Optional: Compute class weights for imbalanced data
from sklearn.utils import class_weight
import numpy as np
y_train = np.concatenate([labels.numpy() for _, labels in train_ds])
class_weights = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train
)
class_weights = dict(enumerate(class_weights))

# Further train the best model
history = best_model.fit(
    train_ds,
    epochs=50,  # Train for more epochs
    validation_data=validation_ds,
    class_weight=class_weights,  # Apply class weights if needed
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Evaluate the model on test dataset
loss, accuracy = best_model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:



# Save best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
with open(os.path.join(output_dir, 'best_hyperparameters.json'), 'w') as f:
    json.dump(best_hps.values, f, indent=4)

# Save trials summary
trials_summary = [
    {
        'trial_id': trial.trial_id,
        'hyperparameters': trial.hyperparameters.values,
        'score': trial.score,
        'status': trial.status
    } for trial in tuner.oracle.trials.values()
]
with open(os.path.join(output_dir, 'trials_summary.json'), 'w') as f:
    json.dump(trials_summary, f, indent=4)

# Save training history
with open(os.path.join(output_dir, 'training_history.pkl'), 'wb') as f:
    pickle.dump(history.history, f)

# Save trials history
best_trials = tuner.oracle.get_best_trials(num_trials=5)
trials_history = {trial.trial_id: trial.metrics.get_history() for trial in best_trials}
with open(os.path.join(output_dir, 'trials_history.pkl'), 'wb') as f:
    pickle.dump(trials_history, f)

# Create ZIP archive
shutil.make_archive('/kaggle/working/inceptionv3_models', 'zip', output_dir)
print("All files saved in", output_dir)
print("ZIP archive created at /kaggle/working/inceptionv3_models.zip")

In [ ]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,327,585 (85.17 MB)

 Trainable params: 22,293,153 (85.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [ ]:
# Plot training curves
def smooth_curve(points, factor=0.8):
    smoothed = []
    for point in points:
        if smoothed:
            smoothed.append(smoothed[-1] * factor + point * (1 - factor))
        else:
            smoothed.append(point)
    return smoothed

plt.plot(smooth_curve(history.history['accuracy']), color='red', label='train')
plt.plot(smooth_curve(history.history['val_accuracy']), color='blue', label='validation')
plt.title('Model Accuracy')
plt.legend()
plt.show()

plt.plot(smooth_curve(history.history['loss']), color='red', label='train')
plt.plot(smooth_curve(history.history['val_loss']), color='blue', label='validation')
plt.title('Model Loss')
plt.legend()
plt.show()

Found 2142 files belonging to 2 classes.
Found 420 files belonging to 2 classes.


In [ ]:
# Load the Validation Dataset

from tensorflow.keras.utils import image_dataset_from_directory

test_dir = '/content/Fold1/Fold1/Fold1/Test'

test_ds = image_dataset_from_directory(
    directory=val_dir,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)


In [ ]:
# Evaluate on test set
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
# Classification report and AUC
y_true = []
y_pred_probs = []
for images, labels in test_ds:
    preds = model.predict(images).flatten()
    y_true.extend(labels.numpy())
    y_pred_probs.extend(preds)

class_names = test_ds.class_names
print(classification_report(y_true, (np.array(y_pred_probs) > 0.5).astype(int), target_names=class_names))
auc = roc_auc_score(y_true, y_pred_probs)
print(f"AUC-ROC: {auc:.4f}")

Epoch 1/20
14/67 ━━━━━━━━━━━━━━━━━━━━ 14:01 16s/step - accuracy: 0.6053 - loss: 0.6710

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, (np.array(y_pred_probs) > 0.5).astype(int))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_true = []
y_pred = []

for images, labels in validation_ds:
    preds = model.predict(images).flatten()
    pred_labels = (preds > 0.5).astype(int)
    y_true.extend(labels.numpy())
    y_pred.extend(pred_labels)
class_name=["monkeypox","others"]
class_names = validation_ds.class_names  # ['monkeypox', 'others']

print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
# Load the Validation Dataset

from tensorflow.keras.utils import image_dataset_from_directory

test_dir = '/content/Fold1/Fold1/Fold1/Test'

test_ds = image_dataset_from_directory(
    directory=test_dir,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224)
)
# Normalize images
def preprocess(image, label):
    image = tf.cast(image / 255.0, tf.float32)
    return image, label

test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import os
from collections import Counter

train_dir = "/content/Fold1/Fold1/Fold1/Train"
print("Train class counts:")
for label in os.listdir(train_dir):
    count = len(os.listdir(os.path.join(train_dir, label)))
    print(f"{label}: {count}")


Train class counts:
Others: 1162
Monkeypox: 980
